# Steps

In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

## Step 1: Load the data

In [2]:
from langchain.document_loaders import TextLoader

file_path = '../dataset/golden_hymns_of_epictetus.txt'
loader = TextLoader(file_path)
documents = loader.load()

## Step 2: Split the text

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    length_function=len,
    add_start_index=True,
)

text_splitted = text_splitter.split_documents(documents=documents)

## Step 3: Embedding

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding = GoogleGenerativeAIEmbeddings(model='models/text-embedding-004')

## Step 4: VectorStore

In [5]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=text_splitted, embedding=embedding)

## Step 5: Chain

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


template = """
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say 'Ah snap homie, I ain't gonna front. I don't know.`, don't try to make up an answer.
Use three sentences maximum, relevant analogies, and keep the answer as concise as possible.
Use the active voice, and speak directly to the reader using concise language.
{context}
Question: {question}
Helpful Answer:
"""

prompt_template = ChatPromptTemplate.from_template(template)

setup_and_retrieval = RunnableParallel(
    {'context': vectorstore.as_retriever(), 'question': RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt_template | llm | StrOutputParser()

res = chain.invoke("get the names")
print(res)

Ah snap homie, I ain't gonna front. I don't know.
